![AutoSolvate](logo.png)

Autosolvate 
===============


Autsolvate can be run as a command line tool or using a desktop GUI. The following tutorial illustrates the basic usage of Autosolvate in the command line interface (CLI).

There will be two example systems: naphthalene in water with charge method BCC and naphthalene radical in chloroform with charge method RESP. 

Since we are running terminal commands in Jupyter Notebook, we have to prefix `!` before all command line arguments. In the python `Python 3.8 (AUTOSOLVATE)` kernel, `autosolvate` is installed as a bash command. Users can use `conda run -n AUTOSOLVATE autosolvate` command to run `autosolvate`

## First change the directory to your directory

In [1]:
import imolecule
import babel

In [2]:
import os,shutil
user=os.environ["USER"]
os.chdir(f"/home/nanohub/{user}")
# We have to specify the GAMESS version.
os.environ["GAMESS_VERSION"]="2017"

## Load GAMESS module on NanoHub

In [3]:
import hublib.use
%use gamess-20170420

## Change to working directory for first example

In [ ]:
if os.path.exists(os.path.abspath("./neutral")):
    shutil.rmtree(os.path.abspath("./neutral"))
os.mkdir("./neutral")

In [4]:
os.chdir("./neutral")

In [ ]:
!wget https://autosolvate.readthedocs.io/en/latest/_downloads/1406830a8307fc876c90b200c00afed9/naphthalene_neutral.xyz

In [5]:
os.getcwd()

'/home/nanohub/arielgale/tutorial-home/neutral'

# AutoSolvate functions

AutoSolvate has three functions that can be called. They generate the solvent box, run the MD simulations, and extract the micro-solvated clusters from the MD trajectories. They are always called in the following format: <br><br>
`autosolvate boxgen [options]` <br>
`autosolvate mdrun [options]` <br>
`autosolvate clustergen [options]` <br>

To see what the options are for a given function, just use -h or --help 

In [6]:
!conda run -n AUTOSOLVATE autosolvate boxgen -h

AutoSolvate is starting in command line mode!
Running the module to generate solvent box and force field parameters.
Usage: autosolvate boxgen [OPTIONS]
  -m, --main                 solute xyz file
  -s, --solvent              name of solvent
  -o, --output               prefix of the output file names
  -c, --charge               formal charge of solute
  -u, --spinmultiplicity     spin multiplicity of solute
  -g, --chargemethod         name of charge fitting method (bcc, resp)
  -b, --cubesize             size of solvent cube in angstroms
  -r, --srunuse              option to run inside a slurm job
  -q, --qmprogram            name of the quantum chemistry program
  -e, --qmexe                name of the quantum chemistry package executable
  -d, --qmdir                path to the quantum chemistry package
  -a, --amberhome            path to the AMBER molecular dynamics package root directory
  -t, --closeness            Solute-solvent closeness setting
  -l, --solventoff         

# Example 1: neutral naphthalene in water

## Step 1: BOXGEN

The default settings for boxgen are as follows: <br>

<pre>
-s     solvent            water 
-o     output             water_solvated 
-c     charge             0 
-u     multiplicity       1 
-g     chargemethod       bcc 
-b     cubesize           54 A 
</pre>

In [7]:
!conda run -n AUTOSOLVATE autosolvate boxgen -m naphthalene_neutral.xyz -g "bcc" -c 0 -u 1


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 68; net charge: 0

Running: /apps/share64/debian10/anaconda/anaconda-7/envs/AUTOSOLVATE/bin/sqm -O -i sqm.in -o sqm.out

AutoSolvate is starting in command line mode!
Running the module to generate solvent box and force field parameters.
Main/solutexyz naphthalene-neutral.xyz
Chargemethod: bcc
Charge: 0
Spinmultiplicity: 1
['echo', '$AMBERHO

<div class="alert alert-block alert-info">
The important output files from this step are: <br>

water_solvated.inpcrd     input coordinates <br>
water_solvated.pdb        solute + solvent box coordinates <br>
water_solvated.prmtop     Amber parameter-topoogy file for the solute <br><br>

These are the input files for the MD simulation in the next step

## Step 2: MDRUN

The MD run function does the MD simulations using Amber. The default settings are set for a production run with final results coming from the QM/MM NVT trajectory: <br>
<pre>
-f     filename           water_solvated 
-t     temperature        300 
-p     pressure           1 
-i     steps mm min       1,000 
-m     steps mm heat      10,000 
-n     steps mm npt       300,000 
-b     steps mm nve       0 
-l     steps qmmm min     250 
-o     steps qmmm heat    1,000 
-s     steps qmmm nvt     10,000 
-v     steps qmmm nve     0 
-k     DFT functional     B3LYP 

-r     srun use           False 
-d     dry run            False
</pre>

The last two options are related to job submission. When using HPC, it is usually preferred to submit the autosolvate command as a slurm job (-r) or to use the dry run setting (-d), which will generate submit scripts but not run them.

When testing AutoSolvate or for running examples, much smaller runs only using MM is more suitable

In [8]:
!conda run -n AUTOSOLVATE autosolvate mdrun -f water_solvated -i 100 -m 100 -n 100 -l 0 -o 0 -s 0 -v 0

AutoSolvate is starting in command line mode!
Running the module to automatically run MD simulations of solvated structure.
Filename: water_solvated
Steps MM min: 100
Steps MM heat: 100
Steps MM NPT: 100
Steps QMMM min: 0
Steps QMMM heat: 0
Steps QMMM NPT: 0
Steps QMMM NVE: 0
MM Energy minimization
MM Heating
MM NPT equilibration



<div class="alert alert-block alert-info">
This step has completed successfully when the <b>.out</b> and <b>.netcdf</b> (trajectories) files have been generated for each step requested.

## Step 3: CLUSTERGEN

The clustergen function extracts the solute and its surrounding solvent molecules from the designated frames of the MD trajectory. The default settings are as follows: <br>
<pre>
-f     .prmtop file name      NA
-t     .netcdf file name      NA
-a     start frame            0 
-i     frame interval         100 (minimum)
-s     solvent shell size     4
</pre>

In [11]:
!conda run -n AUTOSOLVATE autosolvate clustergen -f water_solvated.prmtop -t water_solvated-mmnpt.netcdf -a 0 -i 10

AutoSolvate is starting in command line mode!
Running the module to extract microsolvated clusters from MD trajectories with solvent box.
Filename: water_solvated.prmtop
Trajectory name: water_solvated-mmnpt.netcdf
startframe to extract: 0
interval to extract: 10
Loading trajectory
selecting center solute
extracting from frames: [0]
calculating distance to all solvent molecules
select solvent molecules
for first frame selected 27 solvent molecules
saving xyz

/apps/share64/debian10/anaconda/anaconda-7/envs/AUTOSOLVATE/lib/python3.8/site-packages/mdtraj/formats/netcdf.py:154: UserWarning: Warning: The 'netCDF4' Python package is not installed. MDTraj is using the 'scipy' implementation to read and write netCDF files,which can be significantly slower.
For improved performance, consider installing netCDF4. See installation instructions at:
https://unidata.github.io/netcdf4-python/#quick-install
  warnings.warn(warning_message)



<div class="alert alert-block alert-info">
This step is completed when the <b>cutoutn-0.xyz</b> files have been generated.

## Step 4: Visualize the Output

In [14]:
imolecule.draw("./water_solvated-cutoutn-0.xyz",size=(250, 250), camera_type="orthographic")

# Example 2: radical napthalene in chloroform

### Change working directory for example 2

In [ ]:
os.chdir(f"/home/nanohub/{user}")
if os.path.exists(os.path.abspath("./radical")):
    shutil.rmtree(os.path.abspath("./radical"))
os.mkdir("./radical")

In [15]:
os.chdir("./radical")

In [ ]:
!wget https://autosolvate.readthedocs.io/en/latest/_downloads/48816e2419a2b9b6e5b94926f5bb051e/naphthalene_radical.xyz

In [16]:
os.getcwd()

'/home/nanohub/arielgale/tutorial-home/radical'

## Step 1: BOXGEN

For this system, we are changing many of the default settings, so we will use the option flags to reset them:
<pre>
  --solvent        chloroform 
  --output         chcl3_solvated
  --charge         1 
  --multiplicity   2
  --chargemethod   resp </pre>

<div class="alert alert-block alert-danger">
Open shell systems must use the RESP charge fitting method! This requires access to Gaussian or GAMESS. 

Please make sure to delete `resp_scr` directory before running the step-1 command

In [17]:
if os.path.exists(os.path.join(os.getcwd(),'resp_scr')):
    shutil.rmtree(os.path.join(os.getcwd(),'resp_scr'))

In [19]:
!conda run -n AUTOSOLVATE autosolvate boxgen -m naphthalene_radical.xyz -s chloroform -c 1 -u 2 -o chcl3_solvated -d "/apps/share64/debian10/gamess/gamess-20170420/bin" -q "gamess" -g "resp"


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.





Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for ac File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences 

## Step 2: MDRUN

We still use the example run settings instead of production here. <br>
Make sure that the filename is updated!

In [ ]:
!conda run -n AUTOSOLVATE autosolvate mdrun -f ch3cl_solvated -i 100 -m 100 -n 100 -l 0 -o 0 -s 0 -v 0

## Step 3: CLUSTERGEN

Nothing changes here other than the filename

In [ ]:
!conda run -n AUTOSOLVATE autosolvate clustergen -f ch3cl_solvated.prmtop -t ch3cl_solvated-mmnpt.netcdf -a 0 -i 10

## Step 4: Visualize the Output

In [ ]:
imolecule.draw("./water_solvated-cutoutn-0.xyz",size=(250, 250), camera_type="orthographic")